In [26]:
import cv2
import os
import base64
import requests
import pickle
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [16]:
#ส่วนนี้เป็นการเชื่อมต่อข้อมูล HOG
url = "http://localhost:8080/api/gethog"


def img2vec(img):
    v, buffer = cv2.imencode(".jpg", img)
    img_str = base64.b64encode(buffer)
    # print(img_str)
    data = "image data,"+str.split(str(img_str),"'")[1]
    # print(data)
    data = data.split(",")[1]
    response = requests.get(url, json={"img":data})
    return response.json()["Hog"]


In [19]:
#นำข้อมูล Train มาสร้าง pk
list_data = []
path = "dataset\\train"

i=0
for subfolder in os.listdir(path):
    # print(subfolder)
    for f in os.listdir(os.path.join(path,subfolder)):
        img = cv2.imread(os.path.join(path,subfolder)+"\\"+f)
        img_hog = img2vec(img)
        img_hog.append(i)
        list_data.append(img_hog)
        # print(i,os.path.join(path,subfolder)+"\\"+f)
        # print(os.path.join(path,subfolder)+"\\"+f)
    i=i+1
write_path = "carTrainModel.pkl"
pickle.dump(list_data, open(write_path,"wb"))
print("data preparation is done")

data preparation is done


In [20]:
#นำข้อมูล Test มาสร้าง pk
list_data = []
path = "dataset\\test"

i=0
for subfolder in os.listdir(path):
    # print(subfolder)
    for f in os.listdir(os.path.join(path,subfolder)):
        img = cv2.imread(os.path.join(path,subfolder)+"\\"+f)
        img_hog = img2vec(img)
        img_hog.append(i)
        list_data.append(img_hog)
    i=i+1
write_path = "carTestModel.pkl"
pickle.dump(list_data, open(write_path,"wb"))
print("data preparation is done")

data preparation is done


In [22]:
#ทำการ load flie ขอมูลที่สร้าง
carVectors = pickle.load(open('carTrainModel.pkl','rb'))
carVectors_np = np.array(carVectors)
X_train = carVectors_np[:,0:-1]
Y_train = carVectors_np[:,-1]

In [23]:
carVectors = pickle.load(open('carTestModel.pkl','rb'))
carVectors_np = np.array(carVectors)
X_test = carVectors_np[:,0:-1]
Y_test = carVectors_np[:,-1]

In [27]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,Y_train)

y_pred = clf.predict(X_test)

In [28]:
print("Accuracy:",metrics.accuracy_score(Y_test , y_pred))

Accuracy: 0.45633456334563344


In [29]:
print("Confusion Matrix:", metrics.confusion_matrix(Y_test , y_pred))

Confusion Matrix: [[81 13 12 33 17 18 25]
 [ 5 38  3  8  4  1  8]
 [15  5 27  4  4  6 14]
 [26  5  8  7  7  4 17]
 [16  7  2  0 53  2 22]
 [ 8  2 10  7  2 69  8]
 [23 21 16 10 19  5 96]]


In [30]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(3352, 8100)
(813, 8100)
(3352,)
(813,)


In [31]:
#นำmodel ที่เทรนแล้ว มาสร้าง model
path_model = "carBandModel.pk"
pickle.dump(clf, open(path_model,"wb"))